In [1]:
import tensorflow as tf
import os

from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment

from models import StateAutoEncoder
from environments import StateEncoder
from train_sae import generate_dataset
from utils import Normalizer
from train_options import collect_data, compute_avg_reward, get_prepare
from models import OptionHierachy

In [2]:
dir_path = os.getcwd()
model_save_dir = dir_path+"/saved/sae"
checkpoint_path = dir_path+"/checkpoints/sae/"
n_epochs = 100
steps_per_epoch = 10000
batch_size = 64
normalizer = Normalizer(0, 499)
env_name = 'Taxi-v3'

# dataset = generate_dataset(env_name, 200, normalizer)

In [3]:
# train_dataset = dataset.shard(num_shards=2, index=0).shuffle(buffer_size=1024).repeat().batch(
#         batch_size, drop_remainder=True)
# eval_dataset = dataset.shard(num_shards=2, index=1).shuffle(buffer_size=1024).repeat().batch(
#         batch_size, drop_remainder=True)

In [4]:
state_autoencoder = StateAutoEncoder(n_epochs, steps_per_epoch, 12, normalize=True, normalizer=normalizer)
state_autoencoder.use_checkpoints(checkpoint_path)

Restoring from c:\Projects\LHLSA thesis/checkpoints/sae/ckpt


In [5]:
encoding = state_autoencoder.encode([[100], [200], [301], [499]])
print(encoding)
decoding = state_autoencoder.decode(encoding)
print(decoding)

[[1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1.
  1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]
tf.Tensor(
[[ 91.75054]
 [177.45905]
 [284.52228]
 [488.73895]], shape=(4, 1), dtype=float32)


In [6]:
# state_autoencoder.compile()
# history = state_autoencoder.fit(train_dataset)
# state_autoencoder.save(args.savedir)

In [7]:
num_iterations = 20000  # @param {type:"integer"}
initial_collect_steps = 1000  # @param {type:"integer"}
collect_steps_per_iteration = 1  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [8]:
train_py_env = StateEncoder(train_py_env, state_autoencoder)
train_env = tf_py_environment.TFPyEnvironment(train_py_env)

In [9]:
# print(train_env.time_step_spec())
# print(train_env.reset())

In [10]:
options_agent = OptionHierachy(
    batch_size,
    train_env.action_spec(),
    train_env.time_step_spec(),
    num_iterations,
    learning_rate=learning_rate
)

replay_buffer = options_agent.get_replay_buffer(
    replay_buffer_max_length
)

dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2
)
iterator = iter(dataset)

prepare = get_prepare(train_env.time_step_spec())

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [ ]:
avg_reward = compute_avg_reward(
    eval_env, options_agent, num_eval_episodes)
print(avg_reward)
avg_reward_history = [avg_reward]

In [ ]:
collect_data(
    train_env, options_agent, replay_buffer,
    initial_collect_steps, prepare=prepare
)

iterator.next()